In [1]:
! pip install transformers
! pip install gdown
! pip install torcheval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 12.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 11.4 MB/s eta 0:00:00


In [2]:
from transformers import BertModel, BertConfig, BertTokenizerFast
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torcheval.metrics.functional import multiclass_f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd

from tqdm import tqdm

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
!gdown 1HgaaeoVTFptwIXth-mYfEaoWmoj_SHhX
!gdown 19LzgUlM3417TlG6bmo-eSY5vRAKw2ukR
!gdown 10eghJhKqJi-ZU6FoJrRS26CsuYBWFIuK

Downloading...
From: https://drive.google.com/uc?id=1HgaaeoVTFptwIXth-mYfEaoWmoj_SHhX
To: /content/public_data.zip
100% 92.1k/92.1k [00:00<00:00, 62.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=19LzgUlM3417TlG6bmo-eSY5vRAKw2ukR
To: /content/train_all.tsv
100% 1.54M/1.54M [00:00<00:00, 126MB/s]
Downloading...
From: https://drive.google.com/uc?id=10eghJhKqJi-ZU6FoJrRS26CsuYBWFIuK
To: /content/test-no_labels.tsv
100% 298k/298k [00:00<00:00, 49.9MB/s]


In [5]:
!unzip public_data.zip

Archive:  public_data.zip
  inflating: val_empty.tsv           


In [6]:
import pandas as pd

In [119]:
data = pd.read_csv('train_all.tsv', sep='\t')

In [120]:
data

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17024,"[USER], согласно предписаниям Роспотребнадзора...",-1,-1,1,1,-1,-1
1,17025,О несоблюдении карантинных мер контактными лиц...,-1,-1,1,1,-1,-1
2,17027,"[USER], читайте больше книжек на карантине, мо...",-1,-1,1,1,-1,-1
3,17030,Иди почитай инсту наших городских пабликов где...,-1,-1,1,1,-1,-1
4,17031,"Все контактные лица, которых они обозначили, о...",-1,-1,1,1,-1,-1
...,...,...,...,...,...,...,...,...
6712,34033,"[USER], меня больше заботят врачи, которые вын...",-1,-1,-1,-1,2,1
6713,34034,жрёт и жрёт вирус народ когда вакцина к нам пр...,-1,-1,-1,-1,2,1
6714,34035,"Меня другой вопрос волнует, что ещё убьёт эта ...",-1,-1,-1,-1,1,1
6715,34037,"А потом будут говорить про ""российскую медицин...",-1,-1,-1,-1,2,1


In [121]:
CLASS_NAME = "vaccines" #"quarantine"  "masks"

In [122]:
num = 0
length = []
for text in data.text:
    num += 1
    length.append(len(text))
print(np.median(length)), print(np.mean(length)), print(np.max(length)), print(np.min(length))

92.0
114.22897126693465
2848
25


(None, None, None, None)

In [123]:
data = data[['text', f'{CLASS_NAME}_stance', f'{CLASS_NAME}_argument']]

# Set your model output as categorical and save in new label col
data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])

# Transform your output to numeric
data[f'{CLASS_NAME}_stance'] = data['stance_label'].cat.codes
data[f'{CLASS_NAME}_argument'] = data['argument_label'].cat.codes

<ipython-input-123-ec99ae5c8edc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stance_label'] = pd.Categorical(data[f'{CLASS_NAME}_stance'])
<ipython-input-123-ec99ae5c8edc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['argument_label'] = pd.Categorical(data[f'{CLASS_NAME}_argument'])
<ipython-input-123-ec99ae5c8edc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [124]:
data

,text,vaccines_stance,vaccines_argument,stance_label,argument_label
0,"[USER], согласно предписаниям Роспотребнадзора...",0,0,-1,-1
1,О несоблюдении карантинных мер контактными лиц...,0,0,-1,-1
2,"[USER], читайте больше книжек на карантине, мо...",0,0,-1,-1
3,Иди почитай инсту наших городских пабликов где...,0,0,-1,-1
4,"Все контактные лица, которых они обозначили, о...",0,0,-1,-1
...,...,...,...,...,...
6712,"[USER], меня больше заботят врачи, которые вын...",3,2,2,1
6713,жрёт и жрёт вирус народ когда вакцина к нам пр...,3,2,2,1
6714,"Меня другой вопрос волнует, что ещё убьёт эта ...",2,2,1,1
6715,"А потом будут говорить про ""российскую медицин...",3,2,2,1


In [125]:
train, val = train_test_split(data, test_size=0.15)

In [126]:
model_name = 'DeepPavlov/rubert-base-cased-sentence'

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config=config)

# Load the Transformers BERT model
transformer_model = BertModel.from_pretrained(model_name, config=config)

# Load the MainLayer
# bert = transformer_model.layers[0]

In [127]:
def target_to_torch(target):
    return torch.tensor(target.to_numpy()).to(torch.long)
  
def tokenize(text):
    return tokenizer(
    text=text,
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='pt',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [128]:
val_y_stance = target_to_torch(val[f'{CLASS_NAME}_stance'])
val_y_argument = target_to_torch(val[f'{CLASS_NAME}_argument'])

# Tokenize the input (takes some time)
val_x = tokenize(val['text'].to_list())

In [129]:
y_stance = torch.tensor(train[f'{CLASS_NAME}_stance'].to_numpy()).to(torch.long)
y_argument = torch.tensor(train[f'{CLASS_NAME}_argument'].to_numpy()).to(torch.long)

# Tokenize the input (takes some time)
x = tokenize(train['text'].to_list())

In [130]:
transformer_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [131]:
class MyModel(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.linear1 = nn.Sequential(nn.Linear(in_features=768, out_features=768, bias=True), 
                                    nn.Dropout(0.1), nn.ReLU(), 
                                    nn.Linear(in_features=768, out_features=4, bias=True))
        self.linear2 = nn.Sequential(nn.Linear(in_features=768, out_features=768, bias=True), 
                            nn.Dropout(0.1), nn.ReLU(), 
                            nn.Linear(in_features=768, out_features=4, bias=True))

    def forward(self, x):
        out = self.bert(x)[1]
        out1 = self.linear1(out)
        out2 = self.linear2(out)
        return out1, out2

In [132]:
class CustomDataset(Dataset):
    def __init__(self, data, labels_stance, labels_argument):
        self.data = data
        self.labels_stance = labels_stance
        self.labels_argument = labels_argument

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]
        label_stance = self.labels_stance[idx]
        label_argument = self.labels_argument[idx]
        return sentence, label_stance, label_argument

In [133]:
batch_size = 16
epochs_num = 10
learning_rate = 5e-5

In [134]:
train_dataset = CustomDataset(x['input_ids'], y_stance, y_argument)
val_dataset = CustomDataset(val_x['input_ids'], val_y_stance, val_y_argument)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)#, drop_last=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)#, drop_last=True)

In [135]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [136]:
model = MyModel(transformer_model)
model.to(device)
for param in model.bert.parameters():
    param.requires_grad = False

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01, amsgrad=False)

In [137]:
loss_fn = nn.CrossEntropyLoss()

In [138]:
def train_one_epoch(epoch_index, dataset):
    running_loss = 0.
    running_accuracy = 0.
    stance_acc_total = 0.0
    argument_acc_total = 0.0

    obj_size = 0
    for i, data in enumerate(dataset):
        inputs, labels_stance, labels_argument = data
        output1, output2 = model(inputs.to(device))
        obj_size += len(inputs)
        
        stance_predict = torch.argmax(output1.cpu(), dim=1)
        argument_predict = torch.argmax(output2.cpu(), dim=1)
        stance_acc_total += (labels_stance == stance_predict).sum().item()
        argument_acc_total += (labels_argument == argument_predict).sum().item()

        loss1 = loss_fn(output1.cpu(), labels_stance)
        loss2 = loss_fn(output2.cpu(), labels_argument)
        loss = loss1 + loss2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if not i % 100:
            print(f'current loss: {np.round(running_loss / obj_size, 4)}')
            print('stance_loss:', np.round(loss1.item(), 4), 
                  'argument_loss:', np.round(loss2.item(), 4), end=' ')
            print('stance_acc:', np.round(stance_acc_total / obj_size, 4), 
                  'argument_acc:', np.round(argument_acc_total / obj_size, 4))

    return running_loss / obj_size, stance_acc_total / obj_size, argument_acc_total / obj_size


def val_one_epoch(epoch_index, dataset):
    running_loss = 0.
    running_accuracy = 0.
    stance_acc_total = 0.0
    argument_acc_total = 0.0

    stance_pred = []
    arg_pred = []
    obj_size = 0
    for i, data in enumerate(dataset):
        inputs, labels_stance, labels_argument = data
        output1, output2 = model(inputs.to(device))
        obj_size += len(inputs)

        stance_predict = torch.argmax(output1.cpu(), dim=1)
        stance_pred.append(stance_predict)
        argument_predict = torch.argmax(output2.cpu(), dim=1)
        arg_pred.append(argument_predict)

        stance_acc_total += (labels_stance == stance_predict).sum().item()
        argument_acc_total += (labels_argument == argument_predict).sum().item()

        loss1 = loss_fn(output1.cpu(), labels_stance).item()
        loss2 = loss_fn(output2.cpu(), labels_argument).item()
        loss = loss1 + loss2

        running_loss += loss
        if not i % 100:
            print(f'current loss: {np.round(running_loss / obj_size, 4)}')
            print('stance_loss:', np.round(loss1, 4), 
                  'argument_loss:', np.round(loss2, 4), end=' ')
            print('stance_acc:', np.round(stance_acc_total / obj_size, 4), 
                  'argument_acc:', np.round(argument_acc_total / obj_size, 4))

    return running_loss / obj_size, stance_pred, arg_pred

In [139]:
best_vstance = 0
best_varg = 0

for epoch in range(epochs_num):
    print()
    print(f'EPOCH {epoch+1}')
    model.train()
    avg_loss, stance_acc, arg_acc = train_one_epoch(epoch, train_dataloader)
    print(f'----------------TRAIN LOSS on epoch {epoch+1}:', np.round(avg_loss, 4))
    print('TRAIN stance acc:', np.round(stance_acc, 4), 'arg acc:', np.round(arg_acc, 4))

    model.eval()
    with torch.no_grad():
        avg_vloss, stance_pred, arg_pred = val_one_epoch(epoch, val_dataloader)
    stance_pred = torch.cat(stance_pred, dim=0)
    arg_pred = torch.cat(arg_pred, dim=0)
    stance_f1 = multiclass_f1_score(stance_pred[val_y_stance!=0], val_y_stance[val_y_stance!=0], 
                                    num_classes=4, average="macro").item()
    arg_f1 = multiclass_f1_score(arg_pred[val_y_argument!=0], val_y_argument[val_y_argument!=0], 
                                 num_classes=4, average="macro").item()

    print(f'----------------VAL LOSS on epoch {epoch+1}:', np.round(avg_vloss, 4))

    if np.isclose(best_vstance, stance_f1, 1e-4) and (best_varg < arg_f1):
        best_vstance = stance_f1
        best_varg = arg_f1
        model_path = 'model_{}'.format(epoch+1)
        torch.save(model.state_dict(), model_path)
        print(f'best results: best f1_stance = {best_vstance}, best_f1_arg = {best_varg}')
    elif best_vstance < stance_f1:
        best_vstance = stance_f1
        best_varg = arg_f1
        model_path = 'model_{}'.format(epoch+1)
        torch.save(model.state_dict(), model_path)
        print(f'best results: best f1_stance = {best_vstance}, best_f1_arg = {best_varg}')


EPOCH 1
current loss: 0.1754
stance_loss: 1.3847 argument_loss: 1.4219 stance_acc: 0.25 argument_acc: 0.375
current loss: 0.1121
stance_loss: 0.8208 argument_loss: 0.8421 stance_acc: 0.7364 argument_acc: 0.7296
current loss: 0.1054
stance_loss: 0.6639 argument_loss: 0.5355 stance_acc: 0.7404 argument_acc: 0.7369
current loss: 0.1013
stance_loss: 0.9079 argument_loss: 0.9566 stance_acc: 0.7479 argument_acc: 0.7456
----------------TRAIN LOSS on epoch 1: 0.0997
TRAIN stance acc: 0.7514 arg acc: 0.7495
current loss: 0.1501
stance_loss: 1.227 argument_loss: 1.1746 stance_acc: 0.5625 argument_acc: 0.5625
----------------VAL LOSS on epoch 1: 0.0949
best results: best f1_stance = 0.21347357332706451, best_f1_arg = 0.21347357332706451

EPOCH 2
current loss: 0.0712
stance_loss: 0.6348 argument_loss: 0.504 stance_acc: 0.8125 argument_acc: 0.8125
current loss: 0.0961
stance_loss: 0.291 argument_loss: 0.2874 stance_acc: 0.7512 argument_acc: 0.7512
current loss: 0.0946
stance_loss: 0.6475 argument_

In [140]:
for param in model.parameters():
    param.requires_grad = True

In [142]:
best_vstance = 0
best_varg = 0
epochs_num = 6

for epoch in range(epochs_num):
    print()
    print(f'EPOCH {epoch+1}')
    model.train()
    avg_loss, stance_acc, arg_acc = train_one_epoch(epoch, train_dataloader)
    print(f'----------------TRAIN LOSS on epoch {epoch+1}:', np.round(avg_loss, 4))
    print('TRAIN stance acc:', np.round(stance_acc, 4), 'arg acc:', np.round(arg_acc, 4))

    model.eval()
    with torch.no_grad():
        avg_vloss, stance_pred, arg_pred = val_one_epoch(epoch, val_dataloader)
    stance_pred = torch.cat(stance_pred, dim=0)
    arg_pred = torch.cat(arg_pred, dim=0)
    stance_f1 = multiclass_f1_score(stance_pred[val_y_stance!=0], val_y_stance[val_y_stance!=0], 
                                    num_classes=4, average="macro").item()
    arg_f1 = multiclass_f1_score(arg_pred[val_y_argument!=0], val_y_argument[val_y_argument!=0], 
                                 num_classes=4, average="macro").item()

    print(f'----------------VAL LOSS on epoch {epoch+1}:', np.round(avg_vloss, 4))

    if np.isclose(best_vstance, stance_f1, 1e-4) and (best_varg < arg_f1):
        best_vstance = stance_f1
        best_varg = arg_f1
        model_path = 'model_{}'.format(epoch+1)
        torch.save(model.state_dict(), model_path)
        print(f'best results: best f1_stance = {best_vstance}, best_f1_arg = {best_varg}')
    elif best_vstance < stance_f1:
        best_vstance = stance_f1
        best_varg = arg_f1
        model_path = 'model_{}'.format(epoch+1)
        torch.save(model.state_dict(), model_path)
        print(f'best results: best f1_stance = {best_vstance}, best_f1_arg = {best_varg}')


EPOCH 1
current loss: 0.018
stance_loss: 0.1671 argument_loss: 0.1209 stance_acc: 0.875 argument_acc: 0.9375
current loss: 0.0252
stance_loss: 0.398 argument_loss: 0.0331 stance_acc: 0.9016 argument_acc: 0.9431
current loss: 0.0258
stance_loss: 0.2151 argument_loss: 0.2188 stance_acc: 0.8918 argument_acc: 0.9397
current loss: 0.0263
stance_loss: 0.3145 argument_loss: 0.1945 stance_acc: 0.8864 argument_acc: 0.9363
----------------TRAIN LOSS on epoch 1: 0.0265
TRAIN stance acc: 0.8861 arg acc: 0.9345
current loss: 0.0459
stance_loss: 0.41 argument_loss: 0.3246 stance_acc: 0.75 argument_acc: 0.8125


----------------VAL LOSS on epoch 1: 0.0272
best results: best f1_stance = 0.30022522807121277, best_f1_arg = 0.2321428656578064

EPOCH 2
current loss: 0.0046
stance_loss: 0.0516 argument_loss: 0.0214 stance_acc: 1.0 argument_acc: 1.0
current loss: 0.0236
stance_loss: 0.5297 argument_loss: 0.1352 stance_acc: 0.9053 argument_acc: 0.9375
current loss: 0.0233
stance_loss: 0.4032 argument_loss: 0.4717 stance_acc: 0.9058 argument_acc: 0.9412
current loss: 0.0231
stance_loss: 0.211 argument_loss: 0.129 stance_acc: 0.9103 argument_acc: 0.9421
----------------TRAIN LOSS on epoch 2: 0.0228
TRAIN stance acc: 0.9117 arg acc: 0.9425
current loss: 0.0471
stance_loss: 0.3633 argument_loss: 0.3899 stance_acc: 0.8125 argument_acc: 0.8125


----------------VAL LOSS on epoch 2: 0.027
best results: best f1_stance = 0.3628914952278137, best_f1_arg = 0.22871343791484833

EPOCH 3
current loss: 0.0184
stance_loss: 0.1513 argument_loss: 0.1434 stance_acc: 0.9375 argument_acc: 0.9375
current loss: 0.0166
stance_loss: 0.2272 argument_loss: 0.0671 stance_acc: 0.9394 argument_acc: 0.9616
current loss: 0.0167
stance_loss: 0.0777 argument_loss: 0.047 stance_acc: 0.94 argument_acc: 0.9574


KeyboardInterrupt: ignored

In [105]:
def model_predict(model, data):
    stances = []
    args = []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(test_x['input_ids']), batch_size):
            out1, out2 = model(data[i:i+batch_size])
            stances.append(torch.argmax(out1.cpu(), dim=-1))
            args.append(torch.argmax(out2.cpu(), dim=-1))
    return stances, args

In [90]:
test = pd.read_csv('test-no_labels.tsv', sep='\t')

In [92]:
test_x = tokenize(test['text'].to_list())

In [106]:
stances, args = model_predict(model, test_x['input_ids'].to(device))

In [110]:
stances = torch.cat(stances, dim=0)
args = torch.cat(args, dim=0)

In [113]:
test[f'{CLASS_NAME}_stance_predict'] = stances
test[f'{CLASS_NAME}_argument_predict'] = args
test[f'{CLASS_NAME}_stance_predict'] -= 1
test[f'{CLASS_NAME}_argument_predict'] -= 1

In [115]:
test[['text', f'{CLASS_NAME}_stance_predict', f'{CLASS_NAME}_argument_predict']].to_csv(f"test_predict_{CLASS_NAME}.tsv", sep='\t', index=None)

In [134]:


training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", 
    learning_rate=6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs_num,
    weight_decay=0.01,
    adam_epsilon=1e-08,
    max_grad_norm=1.0,
    warmup_ratio=0.0,
    save_strategy='no',
    report_to='none',)

In [141]:
#from datasets import load_dataset, load_metric
import evaluate

metric = evaluate.load("seqeval")

In [143]:
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=2)

    results = metric.compute(predictions=predictions, references=labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [174]:
model(torch.tensor([[101, 13001, 10636, 10783, 166, 102]]))

tensor([[[-0.3672,  0.0743,  0.1122, -0.2410],
         [-0.3672,  0.0594,  0.1683, -0.2139],
         [-0.3998,  0.0879,  0.0840, -0.2644],
         [-0.3471,  0.0596,  0.0118, -0.1929],
         [-0.3169,  0.0183,  0.1473, -0.3025],
         [-0.3318,  0.0742,  0.1135, -0.2131]]], grad_fn=<ViewBackward0>)

In [172]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=x,#tokenized_datasets["train"],
    eval_dataset=val_x,#tokenized_datasets["test"],
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
input_ids = Input(shape=(256,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

# Then build your model output
stance = Dense(units=len(data.stance_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='stance')(pooled_output)
argument = Dense(units=len(data.argument_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='argument')(pooled_output)
outputs = {'stance': stance, 'argument': argument}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  177853440   ['input_ids[0][0]']              
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                               
                                 768),                                                            
                                 pooler_output=(Non                      

In [29]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=5e-05,
    decay_steps=10000,
    decay_rate=0.95)

optimizer = Adam(learning_rate=lr_schedule, epsilon=1e-08,
    clipnorm=1.0, weight_decay=0.01)

# Set loss and metrics
loss = {'stance': CategoricalCrossentropy(from_logits = True), 'argument': CategoricalCrossentropy(from_logits = True)}
metric = {'stance': CategoricalAccuracy('accuracy'), 'argument': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Fit the model
history = model.fit(
    # x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    x={'input_ids': x['input_ids']},
    y={'stance': y_stance, 'argument': y_argument},
    validation_data=({'input_ids': val_x['input_ids'][:8]}, {'stance': val_y_stance[:8], 'argument': val_y_argument[:8]}),
    batch_size=16,
    epochs=20)

Epoch 1/20
315/315 [==============================] - 357s 957ms/step - loss: 0.8044 - argument_loss: 0.3610 - stance_loss: 0.4434 - argument_accuracy: 0.9067 - stance_accuracy: 0.8569 - val_loss: 3.7211 - val_argument_loss: 1.5235 - val_stance_loss: 2.1976 - val_argument_accuracy: 0.5000 - val_stance_accuracy: 0.2500
Epoch 2/20
315/315 [==============================] - 294s 934ms/step - loss: 1.7383 - argument_loss: 0.8229 - stance_loss: 0.9154 - argument_accuracy: 0.6714 - stance_accuracy: 0.6746 - val_loss: 2.2493 - val_argument_loss: 0.8320 - val_stance_loss: 1.4173 - val_argument_accuracy: 0.5000 - val_stance_accuracy: 0.5000
Epoch 3/20
315/315 [==============================] - 293s 931ms/step - loss: 1.6989 - argument_loss: 0.8041 - stance_loss: 0.8948 - argument_accuracy: 0.6837 - stance_accuracy: 0.6837 - val_loss: 2.3252 - val_argument_loss: 0.8096 - val_stance_loss: 1.5156 - val_argument_accuracy: 0.5000 - val_stance_accuracy: 0.5000
Epoch 4/20
315/315 [====================

In [27]:
import gc
gc.collect()

0

In [30]:
val_results = model.predict(x={'input_ids': val_x['input_ids']})
val[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
val[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)
print(classification_report(val[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0))

53/53 [==============================] - 33s 561ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1173
           1       0.00      0.00      0.00        42
           2       0.62      0.97      0.76       312
           3       0.00      0.00      0.00       153

    accuracy                           0.88      1680
   macro avg       0.40      0.49      0.44      1680
weighted avg       0.80      0.88      0.83      1680



In [31]:
print(classification_report(val[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1173
           1       0.00      0.00      0.00        35
           2       0.84      0.96      0.90       428
           3       0.00      0.00      0.00        44

    accuracy                           0.94      1680
   macro avg       0.46      0.49      0.47      1680
weighted avg       0.90      0.94      0.92      1680



In [17]:
val_results = model.predict(x={'input_ids': val_x['input_ids']})

53/53 [==============================] - 34s 567ms/step


In [18]:
val[f'{CLASS_NAME}_stance_predict'] = val_results['stance'].argmax(axis=-1)
val[f'{CLASS_NAME}_argument_predict'] = val_results['argument'].argmax(axis=-1)

In [19]:
from sklearn.metrics import classification_report

print(classification_report(val[f'{CLASS_NAME}_stance'].values.tolist(), val_results['stance'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1173
           1       0.00      0.00      0.00        42
           2       0.62      0.97      0.76       312
           3       0.00      0.00      0.00       153

    accuracy                           0.88      1680
   macro avg       0.40      0.49      0.44      1680
weighted avg       0.80      0.88      0.83      1680



In [20]:
print(classification_report(val[f'{CLASS_NAME}_argument'].values.tolist(), val_results['argument'].argmax(axis=-1), zero_division=0))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1173
           1       0.00      0.00      0.00        35
           2       0.84      0.96      0.90       428
           3       0.00      0.00      0.00        44

    accuracy                           0.94      1680
   macro avg       0.46      0.49      0.47      1680
weighted avg       0.90      0.94      0.92      1680



In [ ]:
testest_x = tokenizer(
    text=test['text'].to_list(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

test_results = model.predict(x={'input_ids': testest_x['input_ids']})
test[f'{CLASS_NAME}_stance_predict'] = test_results['stance'].argmax(axis=-1)
test[f'{CLASS_NAME}_argument_predict'] = test_results['argument'].argmax(axis=-1)

In [ ]:
test[['text', f'{CLASS_NAME}_stance_predict', f'{CLASS_NAME}_argument_predict']].to_csv(f"test_predict_{CLASS_NAME}.tsv", sep='\t', index=None)

In [ ]:
df1 = pd.read_csv(f"val_predict_{CLASS_NAME}.tsv", sep='\t')
df1.rename(columns={"quarantine_stance_predict": "quarantine_stance",
                    "quarantine_argument_predict": "quarantine_argument"}, inplace=True)